In [ ]:
# pretrained
import transformers
processor = transformers.Wav2Vec2Processor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english", use_fast=True)
model = transformers.AutoModelForAudioClassification.from_pretrained("ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition").cuda()
pred = transformers.pipeline("audio-classification", model=model, tokenizer=processor.tokenizer, feature_extractor=processor.feature_extractor, device=0, return_all_scores=True)

In [ ]:
# pretrained model trying
import librosa
import torch
import glob
audios = glob.glob("cmumosi/Raw/Audio/WAV_16000/Segmented/*.wav")[:100]
outs = []

for a in audios:
    try:
        speech, rate = librosa.load(a, sr=16000)
        speech = torch.Tensor(speech).cuda()
        # https://www.analyticsvidhya.com/blog/2021/02/hugging-face-introduces-the-first-automatic-speech-recognition-model-wav2vec2/
        input_values = processor(speech, sampling_rate = 16000, return_tensors = 'pt').input_values.cuda()
        #features_extracted = processor.feature_extractor(speech)
        logits = model(input_values).logits
        # https://huggingface.co/docs/transformers/v4.15.0/en/model_doc/wav2vec2#transformers.Wav2Vec2Model
        # emotions = ['angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised']
        # predicted_ids = torch.argmax(logits, dim=-1)
        # transcription = emotions[predicted_ids.item()]
        # outs.append(transcription)
    except RuntimeError:
        print("rip") # cuda out of memory for too big
    # clean it up tho
    del speech
    torch.cuda.empty_cache()